In [ ]:
from datetime import datetime
import requests
from algosdk import future, encoding, logic
from contracts.crowdfounding import utils
from algosdk.v2client import algod


In [ ]:
# Test data

private_key = 'ixz3gYNh1EsJszFCgUgrSQlT//fuGZK7hAzdNftlVRZ5rzuM7Zz5tPXLhPWRvpvNYsBwrvyQt+HCyJstOgiSjg=='
address = 'PGXTXDHNTT43J5OLQT2ZDPU3ZVRMA4FO7SILPYOCZCNS2OQISKHIUGJ3SM'


# address need to be founded from the sandbox
money_address = "6B2KSSG2ZWW3X3NNONJ4NNFWJXBHGI3K7VUZHISFKUVP2UEXF22QJDF7WY"
amount = 100000000
print("goal clerk send -a " + str(amount) + " -f " + money_address + " -t " + address)


In [ ]:
query = {'address': address}
response = requests.get('http://localhost:8501/address_info', params=query)
print(response.json())


In [ ]:
# This cell is for testing purpose only

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)

### Creating the contract

API EXAMPLE

 > http://localhost:8501/create_contract?sender=SUBSTITUTE_WITH_PUBLIC_KEY_&pool_name=prova&target=10000&startTime=1653775200&endTime=1653948000


In [ ]:
startTime = int(datetime.timestamp(datetime(2022,5,29)))
endTime = int(datetime.timestamp(datetime(2022,5,30)))
pool_name = "prova_pmz"
target = 12300000 # microAlgo


query = {"sender": address, "pool_name":pool_name,"target":target, "startTime": startTime, "endTime":endTime}
txt_to_sign_encoded = requests.get('http://localhost:8501/create_contract', params=query).json()

In [ ]:
txt_to_sign = encoding.future_msgpack_decode(txt_to_sign_encoded)

signedTxn = txt_to_sign.sign(private_key)

txid = algod_client.send_transaction(signedTxn)
return_conf = utils.inspect_transaction(txid,algod_client)

### SETUP THE CONTRACT

In [ ]:
import json
import pprint

query = {'address': address}
response = requests.get('http://localhost:8501/address_info', params=query)

result_dict = response.json()
lista_app_create_account = result_dict['created-apps']

id = lista_app_create_account[-1]['id'] # questo è l'id della nostra applicazione appena creata 

In [ ]:
from algosdk.constants import MIN_TXN_FEE


parameters = algod_client.suggested_params()
parameters.flat_fee = True
parameters.fee = 4 * MIN_TXN_FEE



fundAppTxn = future.transaction.PaymentTxn(
        sender = address,
        receiver = logic.get_application_address(id),
        amt = 500000,
        sp = algod_client.suggested_params(),
    )


setupTxn = future.transaction.ApplicationNoOpTxn(
        sender = address,
        index=id,
        app_args=[b"setup"],
        sp=algod_client.suggested_params()
)

future.transaction.assign_group_id([fundAppTxn, setupTxn])


signedFundAppTxn = fundAppTxn.sign(private_key)
signedSetupTxn = setupTxn.sign(private_key)

indedd = algod_client.send_transactions([signedFundAppTxn, signedSetupTxn])
utils.inspect_transaction(indedd, algod_client)

In [ ]:
query = {'address': logic.get_application_address(id)}
response = requests.get('http://localhost:8501/address_info', params=query)
print(response.json())